In [0]:
!pip install kagglehub

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ziya07/healthcare-iot-data")

print("Path to dataset files:", path)

Path to dataset files: /home/spark-8972ce90-80a7-4720-b0f2-6a/.cache/kagglehub/datasets/ziya07/healthcare-iot-data/versions/1


In [0]:
import os

# Check what files are inside the dataset folder
print("Files in dataset path:", os.listdir(path))

Files in dataset path: ['healthcare_iot_target_dataset.csv']


In [0]:
import pandas as pd

# Replace with the actual filename you saw in os.listdir(path)
csv_file = os.path.join(path, "healthcare_iot_target_dataset.csv")  

df = pd.read_csv(csv_file)
print("Shape of dataset:", df.shape)
df.head()

Shape of dataset: (200, 13)


,Patient_ID,Timestamp,Sensor_ID,Sensor_Type,Temperature (°C),Systolic_BP (mmHg),Diastolic_BP (mmHg),Heart_Rate (bpm),Device_Battery_Level (%),Target_Blood_Pressure,Target_Heart_Rate,Target_Health_Status,Battery_Level (%)
0,8270,2024-01-01 00:00:00,4,Temperature,36.8,150.0,95.0,84.0,99,140.0,90.0,Unhealthy,80
1,1860,2024-01-01 01:00:00,1,Battery,37.3,150.0,95.0,77.0,84,140.0,90.0,Unhealthy,94
2,6390,2024-01-01 02:00:00,1,Temperature,36.5,140.0,85.0,99.0,93,130.0,80.0,Healthy,84
3,6191,2024-01-01 03:00:00,4,Battery,37.3,120.0,80.0,67.0,95,150.0,90.0,Unhealthy,92
4,6734,2024-01-01 04:00:00,4,Temperature,36.8,90.0,60.0,98.0,97,120.0,80.0,Unhealthy,87


In [0]:
import numpy as np

# Repeat dataset 50 times (adjust to reach 4000+ rows)
expanded_df = pd.concat([df] * 50, ignore_index=True)

# Add small random variations to numeric columns
if "Heart_Rate (bpm)" in expanded_df.columns:
    expanded_df["Heart_Rate (bpm)"] = expanded_df["Heart_Rate (bpm)"] + np.random.randint(-5, 6, size=len(expanded_df))

if "Temperature" in expanded_df.columns:
    expanded_df["Temperature"] = expanded_df["Temperature"] + np.random.uniform(-0.5, 0.5, size=len(expanded_df))

if "Device_Battery_Level (%)" in expanded_df.columns:
    expanded_df["Device_Battery_Level (%)"] = expanded_df["Device_Battery_Level (%)"] + np.random.randint(-1, 2, size=len(expanded_df))

if "Battery_Level (%)" in expanded_df.columns:
    expanded_df["Battery_Level (%)"] = expanded_df["Battery_Level (%)"] + np.random.randint(-2, 3, size=len(expanded_df))

if "Target_Heart_Rate" in expanded_df.columns:
    expanded_df["Target_Heart_Rate"] = expanded_df["Target_Heart_Rate"] + np.random.randint(-3, 4, size=len(expanded_df))

if "Target_Blood_Pressure" in expanded_df.columns:
    expanded_df["Target_Blood_Pressure"] = expanded_df["Target_Blood_Pressure"] + np.random.randint(-5, 6, size=len(expanded_df))

print("Expanded shape:", expanded_df.shape)
expanded_df.tail()

Expanded shape: (10000, 13)


,Patient_ID,Timestamp,Sensor_ID,Sensor_Type,Temperature (°C),Systolic_BP (mmHg),Diastolic_BP (mmHg),Heart_Rate (bpm),Device_Battery_Level (%),Target_Blood_Pressure,Target_Heart_Rate,Target_Health_Status,Battery_Level (%)
9995,7546,2024-01-09 03:00:00,4,Blood Pressure,37.0,130.0,85.0,81.0,81,140.0,72.0,Healthy,87
9996,2986,2024-01-09 04:00:00,1,Heart Rate,37.3,130.0,80.0,77.0,91,115.0,77.0,Unhealthy,94
9997,9338,2024-01-09 05:00:00,4,Heart Rate,36.9,140.0,90.0,84.0,91,116.0,81.0,Healthy,94
9998,3911,2024-01-09 06:00:00,2,Temperature,37.1,90.0,60.0,80.0,96,118.0,80.0,Healthy,83
9999,2734,2024-01-09 07:00:00,3,Blood Pressure,37.4,150.0,95.0,66.0,82,128.0,82.0,Unhealthy,86


In [0]:
# Save locally (driver node)
expanded_df.to_csv("Healthcare_IoT_Expanded.csv", index=False)


In [0]:
# Read the CSV from your volume
spark_df = spark.read.csv(
    "dbfs:/Volumes/workspace/default/my_dataset/Healthcare_IoT_Expanded.csv",
    header=True,
    inferSchema=True
)

print("Row count:", spark_df.count())
spark_df.show(5, truncate=False)

Row count: 10000
+----------+-------------------+---------+-----------+----------------+------------------+-------------------+----------------+------------------------+---------------------+-----------------+--------------------+-----------------+
|Patient_ID|Timestamp          |Sensor_ID|Sensor_Type|Temperature (°C)|Systolic_BP (mmHg)|Diastolic_BP (mmHg)|Heart_Rate (bpm)|Device_Battery_Level (%)|Target_Blood_Pressure|Target_Heart_Rate|Target_Health_Status|Battery_Level (%)|
+----------+-------------------+---------+-----------+----------------+------------------+-------------------+----------------+------------------------+---------------------+-----------------+--------------------+-----------------+
|8270      |2024-01-01 00:00:00|4        |Temperature|36.8            |150.0             |95.0               |89.0            |98                      |137.0                |93.0             |Unhealthy           |78               |
|1860      |2024-01-01 01:00:00|1        |Battery    |3

In [0]:
spark_df.printSchema()
spark_df.describe().show()

root
 |-- Patient_ID: integer (nullable = true)
 |-- Timestamp: timestamp (nullable = true)
 |-- Sensor_ID: integer (nullable = true)
 |-- Sensor_Type: string (nullable = true)
 |-- Temperature (°C): double (nullable = true)
 |-- Systolic_BP (mmHg): double (nullable = true)
 |-- Diastolic_BP (mmHg): double (nullable = true)
 |-- Heart_Rate (bpm): double (nullable = true)
 |-- Device_Battery_Level (%): integer (nullable = true)
 |-- Target_Blood_Pressure: double (nullable = true)
 |-- Target_Heart_Rate: double (nullable = true)
 |-- Target_Health_Status: string (nullable = true)
 |-- Battery_Level (%): integer (nullable = true)

+-------+------------------+------------------+-----------+-----------------+------------------+-------------------+-----------------+------------------------+---------------------+-----------------+--------------------+-----------------+
|summary|        Patient_ID|         Sensor_ID|Sensor_Type| Temperature (°C)|Systolic_BP (mmHg)|Diastolic_BP (mmHg)| Heart_Ra

In [0]:
from pyspark.sql import functions as F

clean_df = (
    spark_df
    .dropna()
    .withColumn("Heart_Rate (bpm)", F.col("Heart_Rate (bpm)").cast("double"))
    .withColumn("Temperature (°C)", F.col("Temperature (°C)").cast("double"))
    .withColumn("Device_Battery_Level (%)", F.col("Device_Battery_Level (%)").cast("double"))
    .withColumn("Battery_Level (%)", F.col("Battery_Level (%)").cast("double"))
)

display(clean_df.limit(5))

Patient_ID,Timestamp,Sensor_ID,Sensor_Type,Temperature (°C),Systolic_BP (mmHg),Diastolic_BP (mmHg),Heart_Rate (bpm),Device_Battery_Level (%),Target_Blood_Pressure,Target_Heart_Rate,Target_Health_Status,Battery_Level (%)
8270,2024-01-01T00:00:00.000Z,4,Temperature,36.8,150.0,95.0,89.0,98.0,137.0,93.0,Unhealthy,78.0
1860,2024-01-01T01:00:00.000Z,1,Battery,37.3,150.0,95.0,78.0,85.0,143.0,92.0,Unhealthy,92.0
6390,2024-01-01T02:00:00.000Z,1,Temperature,36.5,140.0,85.0,99.0,94.0,127.0,82.0,Healthy,86.0
6191,2024-01-01T03:00:00.000Z,4,Battery,37.3,120.0,80.0,66.0,94.0,148.0,89.0,Unhealthy,90.0
6734,2024-01-01T04:00:00.000Z,4,Temperature,36.8,90.0,60.0,98.0,98.0,122.0,83.0,Unhealthy,87.0


In [0]:
feature_df = (
    clean_df
    .withColumn("HeartRate_Deviation", F.col("Heart_Rate (bpm)") - F.col("Target_Heart_Rate"))
    .withColumn("Battery_Drop", F.col("Target_Blood_Pressure") - F.col("Battery_Level (%)"))
)

In [0]:
anomaly_df = (
    feature_df
    .withColumn(
        "Anomaly",
        (
            (F.col("Heart_Rate (bpm)") > 180) |
            (F.col("Heart_Rate (bpm)") < 40) |
            (F.col("Temperature (°C)") > 39.5) |
            (F.col("Temperature (°C)") < 35.0) |
            (F.col("Device_Battery_Level (%)") < 10)
        ).cast("int")
    )
)

display(
    anomaly_df.select(
        "Heart_Rate (bpm)",
        "Temperature (°C)",
        "Device_Battery_Level (%)",
        "Anomaly"
    ).limit(10)
)

Heart_Rate (bpm),Temperature (°C),Device_Battery_Level (%),Anomaly
89.0,36.8,98.0,0
78.0,37.3,85.0,0
99.0,36.5,94.0,0
66.0,37.3,94.0,0
98.0,36.8,98.0,0
96.0,36.6,87.0,0
69.0,37.04055555555555,90.0,0
94.0,37.1,85.0,0
101.0,37.4,79.0,0
87.0,37.2,99.0,0


In [0]:
# Install PyOD
%pip install pyod

import pandas as pd
from pyod.models.iforest import IForest

# Convert Spark DataFrame to Pandas DataFrame
pdf = clean_df.select(
    "Heart_Rate (bpm)",
    "Temperature (°C)",
    "Device_Battery_Level (%)",
    "Battery_Level (%)"
).toPandas()

# Fit Isolation Forest
clf = IForest(contamination=0.05)
clf.fit(pdf)

# Predict anomalies
pdf['anomaly'] = clf.predict(pdf)

# Convert back to Spark DataFrame for display
result_df = spark.createDataFrame(pdf)
display(result_df.limit(10))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 MB 17.9 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


/databricks/python/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(


Heart_Rate (bpm),Temperature (°C),Device_Battery_Level (%),Battery_Level (%),anomaly
89.0,36.8,98.0,78.0,1
78.0,37.3,85.0,92.0,0
99.0,36.5,94.0,86.0,0
66.0,37.3,94.0,90.0,0
98.0,36.8,98.0,87.0,0
96.0,36.6,87.0,85.0,0
69.0,37.04055555555555,90.0,82.0,0
94.0,37.1,85.0,86.0,0
101.0,37.4,79.0,81.0,1
87.0,37.2,99.0,84.0,0


In [0]:
anomaly_df_clean = anomaly_df.selectExpr(
    "cast(`Temperature (°C)` as double) as Temperature_C",
    "cast(`Systolic_BP (mmHg)` as double) as Systolic_BP_mmHg",
    "cast(`Diastolic_BP (mmHg)` as double) as Diastolic_BP_mmHg",
    "cast(`Heart_Rate (bpm)` as double) as Heart_Rate_bpm",
    "cast(`Device_Battery_Level (%)` as double) as Device_Battery_Level_pct",
    "cast(`Battery_Level (%)` as double) as Battery_Level_pct"
    # Add other columns as needed, renaming them similarly
)

anomaly_df_clean.write.mode("overwrite").saveAsTable("workspace.default.iot_anomalies")

In [0]:
stream_df = (
    spark.readStream
    .schema(clean_df.schema)
    .option("maxFilesPerTrigger", 1)
    .csv("dbfs:/Volumes/workspace/default/my_dataset/")
)

stream_anomaly = (
    stream_df
    .withColumn("Anomaly",
        ( (F.col("Heart_Rate (bpm)") > 180) |
          (F.col("Heart_Rate (bpm)") < 40) |
          (F.col("Temperature (°C)") > 39.5) |
          (F.col("Temperature (°C)") < 35.0) |
          (F.col("Device_Battery_Level (%)") < 10) )
        .cast("int")
    )
)

query = (
    stream_anomaly.writeStream
    .format("console")
    .outputMode("append")
    .option(
        "checkpointLocation",
        "dbfs:/Volumes/workspace/default/streaming_checkpoints/iot_anomaly_console"
    )
    .trigger(once=True)
    .start()
)

